In [18]:
import numpy as np
import pandas as pd

In [19]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [20]:
train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

In [21]:
train.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,6,1,0,0,0,0,7,0,...,0,0,0,0,0,0,2,0,0,Class_6
1,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,0,0,0,0,0,1,0,3,0,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,0,0,7,0,1,5,2,2,0,1,...,0,4,0,2,2,0,4,3,0,Class_8
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [22]:
import missingno as msno
# msno.matrix(df=train, figsize=(20,14), color=(0.5,0,0))

In [23]:
train_x = train.drop(['target'], axis=1)
train_y = train['target']

test_x = test.copy()

In [24]:
train_x.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,0,0,6,1,0,0,0,0,7,0,...,3,0,0,0,0,0,0,2,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,3,0,0,...,8,0,0,0,0,1,0,0,0,0
3,0,0,7,0,1,5,2,2,0,1,...,0,0,4,0,2,2,0,4,3,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
train_y.head()

0    Class_6
1    Class_6
2    Class_2
3    Class_8
4    Class_2
Name: target, dtype: object

In [26]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y)
train_y = pd.Series(train_y)

In [27]:
train_y.head()

0    5
1    5
2    1
3    7
4    1
dtype: int64

In [28]:
import lightgbm as lgb
#import optuna.integration.lightgbm as lgb
from sklearn.metrics import log_loss

In [43]:
from sklearn.model_selection import train_test_split
tr_x, va_x, tr_y, va_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True, random_state=42, stratify=train_y) 

lgb_train = lgb.Dataset(tr_x, tr_y)
lgb_eval = lgb.Dataset(va_x, va_y)

params = {'objective': 'multiclass',
                  #'random_state': 10,
                  'metric': 'multi_logloss',
             'num_class': 9,
             'force_row_wise' :True}

params2 = {'objective': 'multiclass',
 'metric': 'multi_logloss',
 #'boosting_type':"dart",
 'num_class': 9,
 'force_row_wise': True,
 'feature_pre_filter': False,
 'lambda_l1': 9.743033661523311,
 'lambda_l2': 2.709400532224499e-07,
 'num_leaves': 33,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 100,
 'num_iterations': 50,
 'early_stopping_round': 5}


In [47]:
best_params, history = {}, []
# train
model = lgb.train(params2,
            lgb_train,
            verbose_eval=False,
            early_stopping_rounds=10,
            num_boost_round=100,
            valid_names=['train', 'valid'], 
            valid_sets=[lgb_train, lgb_eval])

/Users/tokimune-i/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/tokimune-i/.pyenv/versions/anaconda3-2020.11/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
[LightGBM] [Info] Total Bins 3286
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 75
[LightGBM] [Info] Start training from score -3.088122
[LightGBM] [Info] Start training from score -2.102454
[LightGBM] [Info] Start training from score -2.603859
[LightGBM] [Info] Start training from score -3.749957
[LightGBM] [Info] Start training from score -4.178678
[LightGBM] [Info] Start training from score -1.350710
[Lig

In [45]:
va_pred = model2.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

logloss: 1.7482


In [32]:
pred = model.predict(test_x)
print(len(pred))

100000


In [33]:
model.params

{'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 9,
 'force_row_wise': True,
 'feature_pre_filter': False,
 'lambda_l1': 9.743033661523311,
 'lambda_l2': 2.709400532224499e-07,
 'num_leaves': 33,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 100,
 'num_iterations': 50,
 'early_stopping_round': 5}

In [34]:
submission = pd.DataFrame({'id': list(range(200000,300001))})
pred = pd.DataFrame(pred,columns =["Class_1","Class_2","Class_3","Class_4","Class_5","Class_6","Class_7","Class_8","Class_9"])
submission = pd.merge(submission,pred,left_index=True, right_index=True)
submission.to_csv('./sample_submission.csv', index=False)
print('csv output')

csv output
